In [100]:
# Take messy user input, refine, answer

from langgraph.graph import StateGraph
from langgraph.graph import START, END
from typing import TypedDict

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv

In [ ]:
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    return_full_text=False,   
    max_new_tokens=150,      # hard stop
    temperature=0.2,         # kill rambling
    top_p=0.9,
    repetition_penalty=1.2   # stops loops
)

model = ChatHuggingFace(llm=llm)

In [102]:
class llmState(TypedDict):
    input: str
    output: str

In [103]:
def getInput(state: llmState) -> llmState:
    UserInput =  state["input"]
    refined=  model.invoke("Rewrite the following into a clear, concise English question. ""Output ONLY the rewritten question.\n\n"f"{UserInput}").content
    state["input"] = refined
    state["output"] = model.invoke(f"Answer the question clearly and directly in 3–5 sentences.\n\nQuestion: {refined}").content
    return state

In [104]:
graph = StateGraph(llmState)

graph.add_node("getInput", getInput)


graph.add_edge(START, "getInput")
graph.add_edge("getInput", END)

workflow = graph.compile()

In [105]:
query =  {"input": "??mahn whhhat izz prompt engg??"}

print(workflow.invoke(query))



{'input': 'What is prompt engineering?', 'output': 'Prompt engineering is the process of designing and refining text prompts to elicit specific, accurate, and relevant responses from artificial intelligence (AI) models, such as language generators or chatbots. It involves understanding the capabilities and limitations of AI models and crafting prompts that effectively communicate the desired output. By optimizing prompts, users can improve the quality, relevance, and coherence of AI-generated responses. This technique has become increasingly important as AI models become more widespread and are used in various applications, including content creation, customer service, and research.'}
